## Importando Bibliotecas

In [1]:
import tensorflow as tf 
from tensorflow import keras
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Input, Dense
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from keras.optimizers import Adam
from keras.models import Model
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

Using TensorFlow backend.


## Pre-processamento dos dados
**Carregando os dados de treino e teste:**

In [2]:
train_images = pd.read_csv('../fashion-mnist_train.csv')
test_images = pd.read_csv('../fashion-mnist_test.csv')

**Separando as labels do conjuntos:**

In [3]:
train_labels=train_images.loc[:, ['label']]
train_images=train_images.drop(['label'], axis=1)
test_labels=test_images.loc[:, ['label']]
test_images=test_images.drop(['label'], axis=1)

**Normalizando os data-sets:**

In [4]:
train_images = train_images / 255.0
test_images = test_images / 255.0


----------------------
## Construção do modelo base
**Esta rede será utilizada posteriomente para comparações entre os modelos de dimensionalidade reduzida utilizando PCA e autoencoder.**

**Definição da estrutura do modelo por meio do Keras Sequential:**

In [5]:
model = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(784)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 59,850
Trainable params: 59,850
Non-trainable params: 0
_________________________________________________________________


**Compilação do modelo:**

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

**Treino do modelo:**

In [7]:
model.fit(train_images, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 5s 97us/sample - loss: 0.5203 - accuracy: 0.8146 - val_loss: 0.4260 - val_accuracy: 0.8468
Epoch 2/5
48000/48000 [==============================] - 5s 114us/sample - loss: 0.3858 - accuracy: 0.8591 - val_loss: 0.4028 - val_accuracy: 0.8551
Epoch 3/5
48000/48000 [==============================] - 5s 96us/sample - loss: 0.3485 - accuracy: 0.8719 - val_loss: 0.3513 - val_accuracy: 0.8714
Epoch 4/5
48000/48000 [==============================] - 5s 96us/sample - loss: 0.3246 - accuracy: 0.8802 - val_loss: 0.3438 - val_accuracy: 0.8743
Epoch 5/5
48000/48000 [==============================] - 6s 125us/sample - loss: 0.3054 - accuracy: 0.8864 - val_loss: 0.3351 - val_accuracy: 0.8824


**Verificação dos resultados do modelo:**

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 44us/sample - loss: 0.3260 - accuracy: 0.8842
Model - 3 layers - test loss: 32.59755953073501
Model - 3 layers - test accuracy: 88.41999769210815


-----------------------------
## Redução de Dimensionalidade Usando Principal component analysis (PCA)

**Definição do número de componentes:**

In [9]:
pca = PCA(n_components=3)

**Construção dos novos datasets de treino e de teste com dimensões reduzidas usando o PCA:**

In [10]:
train_images_r = pca.fit(train_images).transform(train_images)
test_images_r = pca.fit(test_images).transform(test_images)

**Alteração do tipo dos dados de numpy array para pandas dataframe:**

In [11]:
train_images_r = pd.DataFrame(data=train_images_r)
test_images_r= pd.DataFrame(data=test_images_r)

**Definição da estrutura do modelo por meio do Keras Sequential que receberá os dados com dimensões reduzidas pelo PCA:**

In [12]:
model_r = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(3)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model_r.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                256       
_________________________________________________________________
dense_4 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total params: 9,866
Trainable params: 9,866
Non-trainable params: 0
_________________________________________________________________


**Compilação do modelo:**

In [13]:
model_r.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

**Treino do modelo:**

In [14]:
model_r.fit(train_images_r, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 2s 50us/sample - loss: 0.9775 - accuracy: 0.5980 - val_loss: 0.9035 - val_accuracy: 0.6247
Epoch 2/5
48000/48000 [==============================] - 2s 47us/sample - loss: 0.8924 - accuracy: 0.6280 - val_loss: 0.8794 - val_accuracy: 0.6367
Epoch 3/5
48000/48000 [==============================] - 2s 48us/sample - loss: 0.8738 - accuracy: 0.6361 - val_loss: 0.8718 - val_accuracy: 0.6407
Epoch 4/5
48000/48000 [==============================] - 2s 48us/sample - loss: 0.8674 - accuracy: 0.6397 - val_loss: 0.8609 - val_accuracy: 0.6422
Epoch 5/5
48000/48000 [==============================] - 2s 48us/sample - loss: 0.8623 - accuracy: 0.6426 - val_loss: 0.8660 - val_accuracy: 0.6426


**Verificação dos resultados do modelo:**

In [15]:
test_loss, test_acc = model_r.evaluate(test_images_r, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 22us/sample - loss: 0.8701 - accuracy: 0.6340
Model - 3 layers - test loss: 87.01047845840453
Model - 3 layers - test accuracy: 63.40000033378601


------------------------------------------------------

## Redução de Dimensionalidade Usando Autoencoders

In [16]:
# this is the size of our encoded representations
encoding_dim = 2
input_img = Input(shape=(784,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
encoder = Model(input_img, encoded)
decoded = Dense(784, activation='sigmoid')(encoded)
autoencoder = Model(input_img, decoded)
encoded_input = Input(shape=(encoding_dim,))
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [17]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [18]:
#REMOVER troque o 'adadelta' pelo 'adam'
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [19]:
autoencoder.fit(train_images, train_images,epochs=5,shuffle=True,validation_data=(test_images, test_images))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 6s 103us/step - loss: 0.4380 - val_loss: 0.4080
Epoch 2/5
60000/60000 [==============================] - 6s 105us/step - loss: 0.4010 - val_loss: 0.3940
Epoch 3/5
60000/60000 [==============================] - 6s 103us/step - loss: 0.3904 - val_loss: 0.3854
Epoch 4/5
60000/60000 [==============================] - 6s 105us/step - loss: 0.3832 - val_loss: 0.3794
Epoch 5/5
60000/60000 [==============================] - 6s 107us/step - loss: 0.3789 - val_loss: 0.3768


In [20]:
encoded_imgs_train = encoder.predict(train_images)
encoded_imgs_test = encoder.predict(test_images)


In [21]:
train_images_a = pd.DataFrame(data=encoded_imgs_train)
test_images_a =pd.DataFrame(data=encoded_imgs_test)

In [22]:
model_r = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(2)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model_r.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                192       
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 9,802
Trainable params: 9,802
Non-trainable params: 0
_________________________________________________________________


In [23]:
model_r.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [24]:
model_r.fit(train_images_a, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 68us/sample - loss: 1.2822 - accuracy: 0.4715 - val_loss: 1.1831 - val_accuracy: 0.5287
Epoch 2/5
48000/48000 [==============================] - 3s 66us/sample - loss: 1.1723 - accuracy: 0.5204 - val_loss: 1.1552 - val_accuracy: 0.5290- loss: 1.1758  - ETA: 0s -
Epoch 3/5
48000/48000 [==============================] - 3s 54us/sample - loss: 1.1512 - accuracy: 0.5291 - val_loss: 1.1430 - val_accuracy: 0.5357
Epoch 4/5
48000/48000 [==============================] - 3s 62us/sample - loss: 1.1437 - accuracy: 0.5310 - val_loss: 1.1623 - val_accuracy: 0.5228
Epoch 5/5
48000/48000 [==============================] - 3s 56us/sample - loss: 1.1379 - accuracy: 0.5351 - val_loss: 1.1354 - val_accuracy: 0.5397


In [25]:
test_loss, test_acc = model_r.evaluate(test_images_a, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 35us/sample - loss: 1.1422 - accuracy: 0.5352
Model - 3 layers - test loss: 114.21793071746826
Model - 3 layers - test accuracy: 53.519999980926514


------------------------------------------------------

## Clustering com K-means

Optamos pela utilização das imagens reduzidas por meio do PCA, já que obtiveram por meio deste modelo os melhores resultados de predição na rede neural.

In [26]:
train_images_k = train_images_r
test_images_k = test_images_r

In [27]:
kmean_model = KMeans(n_clusters=10, random_state=10)
y_km = kmean_model.fit(train_images_k)
labels = kmean_model.labels_

In [28]:
cluster_centroids = kmean_model.cluster_centers_

In [29]:
#silhouette_avg = silhouette_score(X = train_images_k, labels = train_labels.values.ravel(), random_state=10)

In [30]:
silhouette_avg = silhouette_score(X = train_images_k, labels = train_labels.values.ravel(), random_state=10)

In [33]:
#metrics.homogeneity_score(train_labels.values.ravel(), labels)

In [34]:
metrics.v_measure_score(train_labels.values.ravel(), labels, beta=1.0)

0.45694727123110823

---------------------------------
## Clustering com o DBScan
